# Lab 8: P-Tuning v2 - Deployment Guide

## 🎯 實驗目標

本 Notebook 探討 **P-Tuning v2 的部署策略與優化方案**。作為深度提示調優方法,P-Tuning v2 在每個 Transformer 層都添加可訓練提示,在技術上**無法合併**到基礎模型中,但其通用性和極致的參數效率使其成為PEFT領域的重要方法。

### 關鍵學習要點
- 理解 P-Tuning v2 深度提示的部署特性
- 掌握多層 Prompt 的優化與管理
- 分析 P-Tuning v2 的推理開銷特性
- 學習跨任務、跨規模的部署策略
- 對比 P-Tuning v2 與其他 PEFT 方法的部署差異

---

## 1. P-Tuning v2 部署特性分析

### 1.1 為什麼 P-Tuning v2 無法合併?

**技術原因**:
- **多層深度修改**: 在**每個** Transformer 層都添加虛擬標記,改變了整個模型的前向傳播結構
- **動態序列拼接**: 每層都需要動態拼接提示向量,無法靜態融合到權重
- **層級依賴性**: 不同層的提示參數相互獨立,無法簡單合併

**與其他 PEFT 方法的架構對比**:

| 方法 | 修改層級 | 是否可合併 | 推理開銷來源 |
|:---|:---|:---|:---|
| **LoRA** | 權重矩陣 | ✅ 可合併 | 無 (合併後) |
| **IA³** | 激活值 | ✅ 可合併 | 無 (合併後) |
| **P-Tuning v1** | 僅輸入層 | ❌ 不可合併 | 低 (MLP + 序列拼接) |
| **P-Tuning v2** | **每個層** | ❌ **不可合併** | **中等** (每層拼接) |
| **Prefix Tuning** | 每層 KV | ❌ 不可合併 | 中等 (KV注入) |
| **Adapter** | 新增模組 | ❌ 不可合併 | 高 (額外層) |

### 1.2 P-Tuning v2 的推理開銷分析

```python
# P-Tuning v2 的前向傳播
for layer in transformer_layers:
    # 1. 拼接該層的提示向量
    layer_prompts = layer_prompt_embeddings  # 形狀: [batch, num_prompts, hidden]
    layer_input = concat([layer_prompts, previous_output])  # ← 每層都需拼接
    
    # 2. 正常的Transformer計算
    layer_output = TransformerLayer(layer_input)
    
    # 3. 移除提示部分,只保留真實序列
    previous_output = layer_output[:, num_prompts:, :]
```

**開銷來源**:
- 每層都需要拼接/截取操作
- 序列長度增加導致注意力計算量增加
- 需要額外的記憶體存儲每層的提示參數

**對比其他方法**:
- **vs P-Tuning v1**: 開銷更大 (每層都有拼接)
- **vs Prefix Tuning**: 類似,但 P-Tuning v2 更簡單 (無MLP)
- **vs Adapter**: P-Tuning v2 開銷較小 (無額外FFN層)

---

## 2. 環境準備與模型載入

In [ ]:
# 導入必要的庫
import torch
import torch.nn as nn
import time
import gc
import os
import json
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
)
from peft import (
    PeftModel,
    PromptTuningConfig,
    get_peft_model,
    TaskType
)
import numpy as np

# 設定設備
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用設備: {device}")

# 設定隨機種子
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

### 2.1 載入訓練好的 P-Tuning v2 模型

In [ ]:
# 模型路徑設定
base_model_name = "bert-base-uncased"
adapter_path = "./bert-ptuning-v2-cola"  # 假設這是訓練好的 P-Tuning v2 路徑
num_labels = 2  # CoLA 是二分類任務

# 載入基礎模型和 tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

base_model = AutoModelForSequenceClassification.from_pretrained(
    base_model_name,
    num_labels=num_labels,
    torch_dtype=torch.float16,
    device_map="auto" if torch.cuda.is_available() else None
)

print(f"基礎模型參數量: {base_model.num_parameters():,}")
print(f"基礎模型記憶體佔用: {base_model.get_memory_footprint() / 1024**2:.2f} MB")

In [ ]:
# 載入 P-Tuning v2 adapter (如果存在)
if os.path.exists(adapter_path):
    # 找到最新的 checkpoint
    checkpoints = [d for d in os.listdir(adapter_path) if d.startswith("checkpoint-")]
    if checkpoints:
        latest_checkpoint = max(
            [os.path.join(adapter_path, d) for d in checkpoints],
            key=os.path.getmtime
        )
        peft_model = PeftModel.from_pretrained(base_model, latest_checkpoint)
        print(f"成功載入 P-Tuning v2 adapter: {latest_checkpoint}")
    else:
        peft_model = PeftModel.from_pretrained(base_model, adapter_path)
        print(f"成功載入 P-Tuning v2 adapter: {adapter_path}")
else:
    # 如果沒有訓練好的 adapter,創建示範配置
    print("未找到訓練好的 adapter,創建示範 P-Tuning v2 配置...")
    
    # P-Tuning v2 使用 PromptTuningConfig,但會應用到每一層
    ptuning_v2_config = PromptTuningConfig(
        task_type=TaskType.SEQ_CLS,
        num_virtual_tokens=100,  # P-Tuning v2 通常用更多的虛擬標記
        prompt_tuning_init="RANDOM"
    )
    
    peft_model = get_peft_model(base_model, ptuning_v2_config)
    print("創建了示範 P-Tuning v2 模型")

# 顯示可訓練參數統計
peft_model.print_trainable_parameters()

## 3. P-Tuning v2 深度提示結構分析

### 3.1 多層 Prompt 參數探索

In [ ]:
def analyze_ptuning_v2_structure(model):
    """分析 P-Tuning v2 的多層結構和參數"""
    ptuning_params = {}
    total_params = 0
    layer_count = 0
    
    for name, param in model.named_parameters():
        if 'prompt' in name.lower() or 'virtual' in name.lower():
            ptuning_params[name] = {
                'shape': param.shape,
                'dtype': param.dtype,
                'requires_grad': param.requires_grad,
                'num_params': param.numel()
            }
            total_params += param.numel()
            layer_count += 1
            
            print(f"P-Tuning v2 參數: {name}")
            print(f"  形狀: {param.shape}")
            print(f"  參數量: {param.numel():,}")
            print()
    
    base_params = sum(p.numel() for p in model.base_model.parameters())
    print(f"=== 統計摘要 ===")
    print(f"檢測到的 Prompt 層數: {layer_count}")
    print(f"總 P-Tuning v2 參數量: {total_params:,}")
    print(f"基礎模型參數量: {base_params:,}")
    print(f"P-Tuning v2 參數佔比: {total_params / base_params * 100:.4f}%")
    
    return ptuning_params, layer_count

ptuning_v2_structure, num_layers = analyze_ptuning_v2_structure(peft_model)

### 3.2 深度提示的計算流程可視化

In [ ]:
def visualize_ptuning_v2_computation(num_virtual_tokens=100, hidden_size=768, num_layers=12):
    """
    可視化 P-Tuning v2 的深度提示計算流程
    """
    print("=== P-Tuning v2 深度提示計算流程 ===")
    print()
    print(f"配置參數:")
    print(f"  虛擬標記數量: {num_virtual_tokens} tokens/layer")
    print(f"  隱藏維度: {hidden_size}")
    print(f"  Transformer 層數: {num_layers}")
    print()
    
    print("計算流程:")
    print()
    
    seq_len = 128  # 假設的輸入序列長度
    
    print(f"層 0 (Embedding 層):")
    print(f"  輸入: [{seq_len}, {hidden_size}]")
    print(f"  拼接虛擬標記: [{num_virtual_tokens}, {hidden_size}]")
    print(f"  輸出: [{num_virtual_tokens + seq_len}, {hidden_size}]")
    print()
    
    for layer_idx in range(1, num_layers + 1):
        if layer_idx <= 2 or layer_idx == num_layers:
            print(f"層 {layer_idx} (Transformer Layer {layer_idx}):")
            print(f"  輸入: [{num_virtual_tokens + seq_len}, {hidden_size}]")
            print(f"  Self-Attention: 包含 {num_virtual_tokens} 個虛擬標記")
            print(f"  FFN 計算")
            print(f"  輸出: [{num_virtual_tokens + seq_len}, {hidden_size}]")
            print()
        elif layer_idx == 3:
            print(f"  ... (中間層省略,結構相同) ...")
            print()
    
    print("最終輸出:")
    print(f"  移除虛擬標記部分")
    print(f"  保留真實序列: [{seq_len}, {hidden_size}]")
    print()
    
    # 參數量計算
    total_params = num_layers * num_virtual_tokens * hidden_size
    print("參數統計:")
    print(f"  每層參數量: {num_virtual_tokens * hidden_size:,}")
    print(f"  總參數量: {total_params:,}")
    
    base_params = 110 * 1e6  # BERT-base
    print(f"  佔基礎模型比例: {total_params / base_params * 100:.4f}%")
    print()
    
    # 計算開銷分析
    attention_overhead = ((num_virtual_tokens + seq_len)**2 / seq_len**2 - 1) * 100
    print("推理開銷分析:")
    print(f"  注意力計算量增加: ~{attention_overhead:.1f}% (每層)")
    print(f"  記憶體增加: ~{total_params * 2 / 1024**2:.2f} MB (FP16)")
    print(f"  額外操作: {num_layers} 次拼接/截取")

visualize_ptuning_v2_computation()

## 4. 推理性能基準測試

In [ ]:
def benchmark_inference(model, tokenizer, test_texts, num_runs=10):
    """推理性能基準測試"""
    model.eval()
    total_time = 0
    results = []
    
    with torch.no_grad():
        for run in range(num_runs):
            start_time = time.time()
            
            for text in test_texts:
                inputs = tokenizer(
                    text,
                    return_tensors="pt",
                    padding=True,
                    truncation=True,
                    max_length=128
                )
                if torch.cuda.is_available():
                    inputs = {k: v.cuda() for k, v in inputs.items()}
                
                outputs = model(**inputs)
                prediction = torch.argmax(outputs.logits, dim=-1).cpu().item()
                
                if run == 0:
                    results.append({'text': text, 'prediction': prediction})
            
            end_time = time.time()
            total_time += (end_time - start_time)
    
    return total_time / num_runs, results

# 測試文本
test_texts = [
    "The cat sat on the mat.",
    "She enjoys reading books.",
    "Books enjoys reading she.",  # 語法錯誤
]

print("=== P-Tuning v2 推理性能測試 ===")
ptuning_v2_time, results = benchmark_inference(peft_model, tokenizer, test_texts)
print(f"平均推理時間: {ptuning_v2_time:.4f} 秒")
print(f"記憶體使用: {peft_model.get_memory_footprint() / 1024**2:.2f} MB")

print("\n=== 與基礎模型對比 ===")
base_time, _ = benchmark_inference(base_model, tokenizer, test_texts)
print(f"基礎模型推理時間: {base_time:.4f} 秒")
print(f"時間開銷: {(ptuning_v2_time - base_time) / base_time * 100:.2f}%")

## 5. 模型保存與部署

In [ ]:
save_path = "./bert-ptuning-v2-deployed"

print(f"=== 保存 P-Tuning v2 模組到: {save_path} ===")
peft_model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print("✅ 保存成功!")

# 計算模組大小
module_size = sum(os.path.getsize(os.path.join(save_path, f)) 
                  for f in os.listdir(save_path) if os.path.isfile(os.path.join(save_path, f)))
print(f"模組大小: {module_size / 1024**2:.2f} MB")

## 6. 部署配置與最佳實踐

In [ ]:
deployment_config = {
    "model_info": {
        "base_model": base_model_name,
        "peft_method": "P-Tuning v2",
        "num_virtual_tokens_per_layer": 100,
        "num_layers": num_layers,
        "mergeable": False,
        "universal_across_tasks": True
    },
    "performance_metrics": {
        "inference_overhead_percent": (ptuning_v2_time - base_time) / base_time * 100,
        "parameter_efficiency": "0.01-0.1%"
    },
    "advantages": [
        "跨任務通用性極強 (NLU + NLG)",
        "規模不變性 - 大模型上效果更好",
        "無需 MLP 編碼器,實現簡單",
        "接近全參數微調的性能"
    ],
    "deployment_notes": {
        "best_for": "需要跨任務、跨規模通用性的場景",
        "inference_overhead": "中等 (~10-15%),但性能收益顯著",
        "recommended_scenarios": [
            "多任務學習",
            "大規模模型微調",
            "需要理解+生成能力"
        ]
    }
}

config_path = os.path.join(save_path, "deployment_config.json")
with open(config_path, 'w', encoding='utf-8') as f:
    json.dump(deployment_config, f, indent=2, ensure_ascii=False)

print("=== P-Tuning v2 部署配置 ===")
print(json.dumps(deployment_config, indent=2, ensure_ascii=False))

## 7. 總結

### P-Tuning v2 的核心價值

**獨特優勢**:
1. **通用性無與倫比**: 同時適用於 NLU 和 NLG 任務
2. **規模不變性**: 模型越大,效果越接近全參數微調
3. **實現簡單**: 無需複雜的 MLP 編碼器
4. **性能卓越**: 僅 0.1% 參數達到 99%+ 全參數性能

**部署權衡**:
- ✅ 極致的參數效率和通用性
- ⚠️ 存在推理開銷 (但相比性能收益是值得的)
- ⚠️ 無法合併,需管理多層提示參數

**最佳應用場景**:
- 大規模模型的高效微調
- 需要跨任務通用性
- 追求最優參數效率-性能平衡

In [ ]:
# 清理資源
print("=== 清理資源 ===")
del peft_model, base_model
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
print("✅ 完成")

print("\n🎉 Lab 8 - P-Tuning v2 部署指南完成!")